In [2]:
import pandas as pd
import sqlite3
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path

# Configurar caminhos (ajuste se o notebook não estiver na pasta 'src')
# Assumindo que o notebook está na raiz 'ML Avancado'
DB_PATH = Path() / ".." / "banco de dados" / "crash_bot_historico.db"

# Conectar ao banco de dados e carregar os dados
try:
    with sqlite3.connect(DB_PATH) as conn:
        # Carrega a tabela principal de multiplicadores
        query = """
            SELECT timestamp, multiplicador
            FROM multiplicadores_historico
            ORDER BY timestamp ASC
        """
        df = pd.read_sql_query(query, conn)
    
    # Converter 'timestamp' para o formato datetime do pandas
    df['timestamp'] = pd.to_datetime(df['timestamp'])
    
    # Definir o timestamp como o índice (crucial para Time Series)
    df = df.set_index('timestamp').sort_index()

    print(f"Sucesso: {len(df)} rodadas carregadas do banco de dados.")
    print("Últimas 5 rodadabs:")
    display(df.tail())

except Exception as e:
    print(f"Erro ao carregar o banco de dados: {e}")

Sucesso: 3197 rodadas carregadas do banco de dados.
Últimas 5 rodadas:


,multiplicador
timestamp,
2025-10-30 10:12:12.725153,1.19
2025-10-30 10:12:33.600386,1.51
2025-10-30 10:12:52.826573,1.39
2025-10-30 10:13:15.401468,1.84
2025-10-30 10:13:29.893308,1.06


In [4]:
# Importar as ferramentas de ML (se não estiverem na primeira célula)
from sklearn.model_selection import TimeSeriesSplit
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import numpy as np # Necessário para o np.mean

# --- CORREÇÃO: DEFINIR A CONSTANTE FALTANTE ---
TARGET_MULTIPLIER = 2.0
# --- FIM DA CORREÇÃO ---

# --- Recriar as features do learning_engine.py ---
df_features = df.copy()

# 1. Lags
for i in range(1, 4):
    df_features[f'lag_{i}'] = df_features['multiplicador'].shift(i)

# 2. Rolling Window (Janela Móvel)
window_size = 5
rolling_series = df_features['multiplicador'].shift(1).rolling(window=window_size)
df_features[f'rolling_mean_{window_size}'] = rolling_series.mean()
df_features[f'rolling_std_{window_size}'] = rolling_series.std()

# 3. Low Streak
is_low = df_features['multiplicador'].shift(1) < TARGET_MULTIPLIER # Agora vai funcionar
cumulative_lows = is_low.astype(int).cumsum()
streak_breaks = cumulative_lows.where(~is_low)
filled_breaks = streak_breaks.ffill().fillna(0)
df_features['low_streak'] = (cumulative_lows - filled_breaks).astype(int)

# 4. Target (Alvo)
df_features['target'] = (df_features['multiplicador'] >= TARGET_MULTIPLIER).astype(int)

# 5. Limpar NaNs (importante!)
df_features = df_features.dropna()

print(f"{len(df_features)} amostras válidas para treinamento após engenharia de features.")

# 6. Separar X (features) e y (alvo)
features_list = ['lag_1', 'lag_2', 'lag_3', 'rolling_mean_5', 'rolling_std_5', 'low_streak']
X = df_features[features_list]
y = df_features['target']

3192 amostras válidas para treinamento após engenharia de features.


In [6]:
# Passo 4: Rodar a Validação Cruzada (O Teste Real)

# --- CORREÇÃO: Trazer o cálculo do hit_rate_real para esta célula ---
# (Precisamos recalcular a linha de base aqui para comparar no final)
TARGET_MULTIPLIER = 2.0 # Definir a constante que usamos na célula anterior
total_rodadas = len(df)
hits = (df['multiplicador'] >= TARGET_MULTIPLIER).sum()
# Calcula a taxa de acerto real (ex: 0.49)
hit_rate_real = (hits / total_rodadas) 
# --- FIM DA CORREÇÃO ---


# --- Rodar o mesmo treinamento e validação do script ---
N_SPLITS_CV = 5
tscv = TimeSeriesSplit(n_splits=N_SPLITS_CV)
model = RandomForestClassifier(n_estimators=100, random_state=42, class_weight="balanced")
scaler = StandardScaler()

accuracies = []

print("Iniciando Validação Cruzada (TimeSeriesSplit)...")

for split_count, (train_index, test_index) in enumerate(tscv.split(X), 1):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    
    # Escalar os dados (Fit no treino, Transform em ambos)
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    
    # Treinar
    model.fit(X_train_scaled, y_train)
    
    # Prever
    y_pred = model.predict(X_test_scaled)
    
    # Avaliar
    acc = accuracy_score(y_test, y_pred)
    accuracies.append(acc)
    print(f"Split {split_count}/{N_SPLITS_CV} - Acurácia: {acc:.3%}")

print("-" * 30)
# Agora esta linha vai funcionar:
print(f"Taxa de Acerto Real (Sorte/Baseline): {hit_rate_real:.3%}")
print(f"Acurácia Média (Modelo): {np.mean(accuracies):.3%}")

Iniciando Validação Cruzada (TimeSeriesSplit)...
Split 1/5 - Acurácia: 50.188%
Split 2/5 - Acurácia: 53.008%
Split 3/5 - Acurácia: 49.248%
Split 4/5 - Acurácia: 53.195%
Split 5/5 - Acurácia: 47.180%
------------------------------
Taxa de Acerto Real (Sorte/Baseline): 45.668%
Acurácia Média (Modelo): 50.564%


In [7]:
# Passo 5: Experimentação (Engenharia de Features V2)
# Vamos adicionar mais features para ver se a acurácia melhora

TARGET_MULTIPLIER = 2.0
print("Iniciando Engenharia de Features V2 (mais features)...")

df_features_v2 = df.copy()

# 1. Lags (Agora até 10)
for i in range(1, 11): # <-- MUDANÇA: de 4 para 11
    df_features_v2[f'lag_{i}'] = df_features_v2['multiplicador'].shift(i)

# 2. Rolling Window (Agora 5, 10 e 20)
for window_size in [5, 10, 20]: # <-- MUDANÇA: Adicionado 10 e 20
    rolling_series = df_features_v2['multiplicador'].shift(1).rolling(window=window_size)
    df_features_v2[f'rolling_mean_{window_size}'] = rolling_series.mean()
    df_features_v2[f'rolling_std_{window_size}'] = rolling_series.std()

# 3. Low Streak (Mantido)
is_low = df_features_v2['multiplicador'].shift(1) < TARGET_MULTIPLIER
cumulative_lows = is_low.astype(int).cumsum()
streak_breaks = cumulative_lows.where(~is_low)
filled_breaks = streak_breaks.ffill().fillna(0)
df_features_v2['low_streak'] = (cumulative_lows - filled_breaks).astype(int)

# 4. Target (Alvo) (Mantido)
df_features_v2['target'] = (df_features_v2['multiplicador'] >= TARGET_MULTIPLIER).astype(int)

# 5. Limpar NaNs (O dropna() vai limpar mais linhas agora, por causa do lag_10 e rolling_20)
df_features_v2 = df_features_v2.dropna()

print(f"{len(df_features_v2)} amostras válidas para treinamento (V2).")

# 6. Separar X e y (com a nova lista de features)
features_list_v2 = [col for col in df_features_v2.columns if col not in ['multiplicador', 'target']]
X_v2 = df_features_v2[features_list_v2]
y_v2 = df_features_v2['target']

print(f"Número de features V2: {len(features_list_v2)}")

Iniciando Engenharia de Features V2 (mais features)...
3177 amostras válidas para treinamento (V2).
Número de features V2: 17


In [8]:
# Teste de Validação com Features V2

print("Iniciando Validação Cruzada (Features V2)...")

# (Usar X_v2 e y_v2)
for split_count, (train_index, test_index) in enumerate(tscv.split(X_v2), 1):
    X_train, X_test = X_v2.iloc[train_index], X_v2.iloc[test_index]
    y_train, y_test = y_v2.iloc[train_index], y_v2.iloc[test_index]
    
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    
    model.fit(X_train_scaled, y_train)
    y_pred = model.predict(X_test_scaled)
    
    acc = accuracy_score(y_test, y_pred)
    accuracies.append(acc)
    print(f"Split {split_count}/{N_SPLITS_CV} - Acurácia: {acc:.3%}")

print("-" * 30)
print(f"Taxa de Acerto Real (Sorte/Baseline): {hit_rate_real:.3%}")
print(f"Acurácia Média (Modelo V1 - Baseline): 50.564%")
print(f"Acurácia Média (Modelo V2 - Novas Features): {np.mean(accuracies):.3%}")

Iniciando Validação Cruzada (Features V2)...
Split 1/5 - Acurácia: 55.577%
Split 2/5 - Acurácia: 51.796%
Split 3/5 - Acurácia: 52.930%
Split 4/5 - Acurácia: 55.766%
Split 5/5 - Acurácia: 51.229%
------------------------------
Taxa de Acerto Real (Sorte/Baseline): 45.668%
Acurácia Média (Modelo V1 - Baseline): 50.564%
Acurácia Média (Modelo V2 - Novas Features): 52.012%
